In [1]:
@generated function harmonic_mean_gen(I...)
           return harmonic_mean_gen_impl(I...)
end
function harmonic_mean_gen_impl(I...)
           N=length(I)
           ex = :(1 / I[$1])
           for i = 2:N
               ex = :($ex + 1 / I[$i])
           end
           return :(1 / $ex)
end

harmonic_mean_gen_impl (generic function with 1 method)

In [2]:
harmonic_mean_gen_impl(Int, Int, Int, Int, Int)

:(1 / ((((1 / I[1] + 1 / I[2]) + 1 / I[3]) + 1 / I[4]) + 1 / I[5]))

In [3]:
dump(:(2/(x*x*x)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol /
    2: Int64 2
    3: Expr
      head: Symbol call
      args: Array{Any}((4,))
        1: Symbol *
        2: Symbol x
        3: Symbol x
        4: Symbol x
      typ: Any
  typ: Any


In [4]:
using Match

function autodiff(ex::Int64)
    return 0
end


function multiplication(args)
    power_counter=0
    basic_multiplier=1
    for i=1:length(args)
        @match args[i] begin
            :x => power_counter+=1
            n => basic_multiplier*=n
        end
    end
    return :($(basic_multiplier*power_counter) *x^$(power_counter-1))
end

function division(numerator, denumerator)
    power_counter=0
    basic_multiplier=1
    for i=1:length(denumerator)
        @match denumerator[i] begin
            :x => power_counter+=1
            n => basic_multiplier*=n
        end
    end
    return :($(numerator)*$(power_counter) /$(basic_multiplier)* x^$(power_counter+1))
end

function main_division(args)
    @match args[2].args begin
        [*, x...] => return division(args[1], x)
    end
end

function autodiff(ex::Symbol)
    @match ex begin
        :x => return 1
    end    
end


function autodiff(ex::Expr)
    res=Expr(:call, :+, 0, 0)
    arr1=[]
    arr2=[]
    @match ex.args begin
        [:+, x...] => for i=1:length(x)
            res = Expr(:call, :+, res, autodiff(x[i]))
        end    
        [:-, x...] => for i=1:length(x)
            if i==1
                res = Expr(:call, :+, res, autodiff(x[i]))
            else
                res = Expr(:call, :-, res, autodiff(x[i]))
            end    
        end
        [:*, x...] =>
            res = Expr(:call, :+, res, multiplication(x))
        [:/, x...] => 
            res = Expr(:call, :-, res, main_division(x))
    end    
    return res    
end

autodiff (generic function with 3 methods)

In [5]:
p=autodiff(:(2*x*x-4*x*x*x +4*3*x))

:(((0 + 0) + (((0 + 0) + ((0 + 0) + 4 * x ^ 1)) - ((0 + 0) + 12 * x ^ 2))) + ((0 + 0) + 12 * x ^ 0))

In [6]:
p=autodiff(:(1/(x*x)))

:((0 + 0) - ((1 * 2) / 1) * x ^ 3)